In [1]:
import sys
sys.path.append('./models')
from models.BBSNet_model import BBSNetTransformerAttention
from BBSNet_train import setup_gpu, get_model, get_data, main
from BBSNet_test import test
from utils import evaluate, draw_curve

%load_ext autoreload
%autoreload 2

/home/mahdi/Documents/pytorch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mahdi/Documents/pytorch/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


# Train

In [2]:
setup_gpu()
model, optimizer, CE = get_model()
train_loader, test_loader, total_step = get_data()
main(optimizer, model, train_loader, test_loader, CE, total_step)

USE GPU 0


/home/mahdi/Documents/pytorch/lib/python3.12/site-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


load data...
Start train...


/home/mahdi/Documents/pytorch/lib/python3.12/site-packages/torch/nn/_reduction.py:51: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


2025-09-18 18:59:55.112922 Epoch [001/002], Step [0001/2184], Loss1: 1.5663 Loss2: 1.5407
Keyboard Interrupt: save model and exit.
save checkpoints successfully!


KeyboardInterrupt: 

# Use Trained Model

I save the model using this code:
torch.save(model.state_dict(), save_path +
                           'BBSNet_epoch_best.pth')

Now, after a long training period, I want to use it on my test data. So, how can I load this model?

In [ ]:
from models.BBSNet_model import BBSNetTransformerAttention

model = BBSNetTransformerAttention()

# assume your model is called `model`
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")
print(f"Non-trainable parameters: {total_params - trainable_params}")


In [ ]:
test(method='combined_p4_109') 

In [ ]:
from latex_table import main
main()

In [ ]:
evaluate()

In [ ]:
file_path = "PySODEvalToolkit-master/output/results.xlsx"